### Setup the base libraries for data processing

In [ ]:
### Setting up libraries for data processing
import numpy as np
import h5py
import random
import os
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import itertools

## Required for NN
import numpy as np
import torch as T
import os
from pstats import SortKey
import time

import matplotlib.font_manager
#plt.style.use(['dark_background'])
plt.rcParams['figure.figsize'] = [15, 9]
# Set general font size
#plt.rcParams['font.size'] = '16'
plt.rcParams['savefig.dpi'] = 200
# plt.rcParams['font.family'] = 'DeJavu Serif'
#plt.rcParams['font.serif'] = ['Helvetica']
plt.rc('font', family='Dejavu Sans')
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
#mpl.rcParams['figure.dpi'] = 200
#from matplotlib import rc
#rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
#rc('text', usetex=True)
plt_colors = ['cornflowerblue', 'crimson', 'forestgreen', 'mediumorchid']

fontsize = 32

sns.set_theme(style='ticks')
sns.set_context("poster")
plt.style.use('seaborn-deep')
plt.style.use('seaborn-talk')
np.random.seed(42)

# Description of Datasets and scripts

### Dataset 

* ADC\_10kData
    + description: the raw dataset
    + location: data['Data']
    + shape: (1600000, 1473, 2)
        - 320000: 32 (#basis\_states) * 10000 (#records per state)
        - 1473: #samples in each trace where samples are collected in a 2ns rate
        - 2: I and Q respectively
        
* all\_traces.npy
    + description: data extracted from 2019\_ReadoutDataAfterADC\_T0, did some rearrangement
    + shape: (32, 10000, 1473, 2) -> (#basis\_states, #records per state, #samples per record, I/Q)
    
* split\_data/\*
    + description: split all\_traces.npy into train/val/test
    + split: train/val/test is 1950/1050/7000 for each basis state

* matched\_filter.py
    + Matched filtering code used for the HERQULES for reference.

### Creating training, validation, and test splits.

In [ ]:
def get_train_val_and_test_set(trace, y, num_qubits=5, 
                               NUM_TRAIN_VAL = 3000,
                               NUM_TEST = 7000, NUM_VAL_RATIO = 0.35):
    
    NUM_VAL = int(NUM_TRAIN_VAL * NUM_VAL_RATIO)
    NUM_TRAIN = NUM_TRAIN_VAL - NUM_VAL

    ## Data for training, validation and testing is in train_set, val_set and test_set
    ## Accordingly labels in y_train, y_val, y_test
    train_set = []
    val_set = []
    test_set = []
    y_train = []
    y_val = []
    y_test = []
    for i in range(0, 2**num_qubits):
        ind = np.where(y==i)[0]
        train_set.append(trace[ind[:NUM_TRAIN], :, :])
        test_set.append(trace[ind[NUM_TRAIN_VAL:], :, :])
        val_set.append(trace[ind[NUM_TRAIN:NUM_TRAIN_VAL], :, :])
        y_train.append(np.array([i for _ in range(NUM_TRAIN)]))
        y_val.append(np.array([i for _ in range(NUM_VAL)]))
        y_test.append(np.array([i for _ in range(NUM_TEST)]))
    train_set = np.reshape(np.array(train_set), (2**num_qubits * NUM_TRAIN, trace.shape[1], trace.shape[2]))
    val_set = np.reshape(np.array(val_set), (2**num_qubits * NUM_VAL, trace.shape[1], trace.shape[2]))
    test_set = np.reshape(np.array(test_set), (2**num_qubits * NUM_TEST, trace.shape[1], trace.shape[2]))
    y_train = np.reshape(np.array(y_train), (2**num_qubits * NUM_TRAIN))
    y_val = np.reshape(np.array(y_val), (2**num_qubits * NUM_VAL))
    y_test = np.reshape(np.array(y_test), (2**num_qubits * NUM_TEST))
    return tuple((train_set, val_set, test_set)), tuple((np.array(y_train), np.array(y_val), np.array(y_test)))

## Demodulation process and pre_classifer_model data

The demodulation class takes the raw ADC sample file and demodulates the readout signal into the constituent readout tones of the five qubits.

In [ ]:
data_path = './'
file_name = 'ADC_10kData'

class demodulation():
	def __init__(self, data_path = "./", file_name = 'ADC_10kData') -> None:
		self.data_path = data_path
		self.file_name = file_name
        
	def y_classes(self, states_bin, num_records=int(1e5), num_Q=1):
		"""Summary

		Args:
			states_bin (TYPE): Description
			num_records (TYPE, optional): Description
			num_Q (int, optional): Description

		Returns:
			TYPE: Description
		"""
		## assumption that same number of records per state configuration
		## assumption samples are sorted
		Binary 		= np.logspace(0.0, num_Q-1, num=num_Q, base=2.0)
		Sta_help 	= np.zeros((2**num_Q,num_Q))
		for i in range(len(states_bin)):
			Sta_help[i,:] = np.remainder([ord(c) for c in states_bin[i]], 48)

		return np.repeat(np.sum(np.multiply(Sta_help, Binary[::-1]), axis=1), num_records).astype(int)

	def states_config(self, num_Q):
		"""Summary
		
		Args:
			num_Q (TYPE): Description
		
		Returns:
			TYPE: Description
		"""
		## different state configuration -> 2^(# of qubits)
		states_bin = np.array(list(itertools.product([0,1], repeat=num_Q)))
		states_bin_help = list(np.zeros(2**num_Q))
		for i in range(2**num_Q):
			states_bin_help[i] = str(states_bin[i,:])[1::2]
		return np.array(states_bin_help)

	def data_dem(self, states_bin, num_samples, sampling_rate, dFreq, num_Q=1, num_records=1e5, skip=0, DT_Bin=2e-9, NoF=0):
		"""Summary

		Args:
			states_bin (TYPE): Description
			num_samples (TYPE): Description
			sampling_rate (TYPE): Description
			dFreq (TYPE): Demodulation at frequency dFreq
			num_Q (int, optional): Description
			num_records (float, optional): Description
			skip (int, optional): Description
			DT_Bin (float, optional): Description
			NoF (int, optional): Description
		"""
		## digital demodulation at frequency freq_readout
		DDem = self.data_res_dem(sampling_rate, dFreq, num_samples, skip)

		## number of binned time steps
		T_D = int(sampling_rate*DT_Bin)

		if T_D > 1:
			num_samples_bin = int(num_samples/T_D)
			print('Bin length [ns]:', DT_Bin*1e9)
			print('Number of binned data points:', T_D)
			print('Raw number of binned samples:', num_samples_bin)

			DDem_help = DDem.shape
			DD = np.zeros([DDem_help[0],num_samples_bin,2])
			for l in range(num_samples_bin):
				DD[:,l,:] = np.mean(DDem[:,l*T_D:(l+1)*T_D,:], axis = 1)

			print('Binned raw processing data shape starting at t0:', DDem.shape)
		else: 
			num_samples_bin = num_samples
			DD = DDem

		## binned time axis
		t_bin = np.linspace(0,T_D*(num_samples_bin-1)/sampling_rate*1e6,num_samples_bin)

		## class labels (2**num_Q)
		y = self.y_classes(states_bin, num_records, num_Q)

		## creation of digitally demodulated data file
		if bool(NoF):
			hf = h5py.File('%s/DD_10k_f%i_v%i' % (data_path, NoF, 1), 'w')
		else:
			hf = h5py.File('%s/DD_10k%iQ_v%i' % (data_path, num_Q, 1), 'w')
		hf.create_dataset('DD', 	data=DD)
		hf.create_dataset('y', 		data=y)
		hf.create_dataset('t_bin', 	data=t_bin)
		hf.close()


	def data_IQ_dem(self, DataI, DataQ, dt, dFreq, dlen, skip=0):
		"""Summary
		
		Args:
			DataI (TYPE): Description
			DataQ (TYPE): Description
			dt (TYPE): Description
			dFreq (TYPE): Description
			dlen (TYPE): Description
			skip (int, optional): Description
		
		Returns:
			TYPE: Description
		"""
		## digital demodulation of analog demodulated I and Q at frequency dFreq
		help_d 	= DataI.shape
		dlen 	= int(min(dlen, help_d[1]))
		
		# normalization to compensate for IQ mixer offset
		DataI 	= DataI-np.mean(DataI)
		DataQ 	= DataQ-np.mean(DataQ)
		
		# normalization for amplitude error IQ mixer
		CorrFa 	= np.std(DataI)/np.std(DataQ)
		DataQ 	= DataQ*CorrFa
		
		# calculate cos/sin vectors, allow segmenting
		vTime 	= dt*(skip+np.arange(dlen-skip, dtype = float))
		vCos 	= np.cos(2*np.pi*vTime*dFreq)
		vSin 	= np.sin(2*np.pi*vTime*dFreq)
		
		del CorrFa, vTime, dFreq, help_d

		# calculate demodulated I/Q from I/Q outputs of mixer  
		dI 		= 2*(vCos*DataI[:,skip:dlen]+vSin*DataQ[:,skip:dlen])
		dQ 		= 2*(vCos*DataQ[:,skip:dlen]-vSin*DataI[:,skip:dlen])
		return (dI, dQ)

	def data_res_dem(self, sampling_rate, dFreq, dlen, skip=0):
		"""Summary
		
		Args:
			sampling_rate (TYPE): Description
			dFreq (TYPE): Description
			dlen (TYPE): Description
			skip (int, optional): Description
		
		Returns:
			TYPE: Description
		"""
		print(self.data_path, self.file_name)
		## open analog demodulated data file
		hf = h5py.File('%s/%s' % (self.data_path, self.file_name), 'r')
		Data_to_Demod 	= hf.get('Data')
		
		## create digital demodulated data file at frequency freq_readout
		(DDemI, DDemQ) = self.data_IQ_dem(DataI=Data_to_Demod[:,:,0], DataQ=Data_to_Demod[:,:,1], dt=1./sampling_rate, dFreq=dFreq, dlen=dlen, skip=0)
		hf.close()
		return np.stack((DDemI,DDemQ), axis = 2)

### Demodulating the readout signal

The resonator frequencies for the 5 constituent qubits is in `freq_readout`. `DT_bin` defines the binning of readout samples that can be used to define the number of samples that are averaged together to represent one sample of the demodulated signal.

In [ ]:
demod = demodulation()
num_Q 			= 5 		# number of qubits
states_bin 		= demod.states_config(num_Q)
sampling_rate 	= 500*1e6 #Samples/sec
num_samples 	= int(2e-6*sampling_rate)
freq_readout 	= -np.array([-64.729*1e6,-25.366*1e6,24.79*1e6,70.269*1e6,127.282*1e6]) #Hz
num_records 	= int(1e4) #number of repeated acquisitions	
DT_Bin			= 2e-9 # or higher

if freq_readout.size>1:
		for i in range(freq_readout.size):
			demod.data_dem(states_bin, num_samples, sampling_rate, freq_readout[i], num_Q, num_records, 0, DT_Bin, i+1)

## Identifying relaxation traces

In [ ]:
def distance(x0, y0, x1, y1):
    return np.sqrt((x0 - x1)**2 + (y0 - y1)**2)

def get_data(qubit):
    file = './DD_10k_f%i_v1'%(qubit)
    data = h5py.File(file, 'r')
    traces = np.array(data['DD'])
    y = np.array(data['y'])
    t = np.array(data['t_bin'])
    return traces, y, t

getbinary = lambda x, n: format(x, 'b').zfill(n)

def get_traces(num_qubits=5, plot=False, rscale=1, data_type=0):
    qubit_traces = {}
    indices_0 = np.arange(10000)
    one_indices = {2**(qubit - 1):None for qubit in range(1, num_qubits+1)}
    for qubit in range(1, num_qubits+1):
        dem, y, t = get_data(qubit)
        #indices_0 = np.arange(np.array(dem).shape[0])
        traces, ys = get_train_val_and_test_set(dem, y)
        dem = traces[data_type]
        y = ys[data_type]
        del traces
        traces = dem
        # Compute the centroid of the clouds corresponding to 0 and 1
        zero = 0
        one = 2**(qubit - 1)
        i0 = np.mean(traces[np.where(y==zero)[0], :, 0], axis=1)
        q0 = np.mean(traces[np.where(y==zero)[0], :, 1], axis=1)
        i1 = np.mean(traces[np.where(y==one)[0], :, 0], axis=1)
        q1 = np.mean(traces[np.where(y==one)[0], :, 1], axis=1)
        i = np.mean(traces[:, :, 0], axis=1)
        q = np.mean(traces[:, :, 1], axis=1)
        x0 = np.mean(i0)
        y0 = np.mean(q0)
        x1 = np.mean(i1)
        y1 = np.mean(q1)
        midx = (x0 + x1) / 2
        midy = (y0 + y1) / 2
        radius = rscale * distance(x0, y0, x1, y1) / 2 # Radius from the centroid where a trace is considered correct
        traces_0 = traces[np.where(y==zero)[0], :, :]
        traces_1 = traces[np.where(y==one)[0], :, :]
        traces_0 = traces_0[np.where(distance(i0, q0, x0, y0) < radius)[0], :, :]
        traces_1 = traces_1[np.where(distance(i1, q1, x1, y1) < radius)[0], :, :]
        indices_0 = np.intersect1d(indices_0, np.where((y==zero) & (distance(i, q, x0, y0) < radius))[0])
        indices_1 = np.where((y==one) & (distance(i, q, x1, y1) < radius))[0]
        
        # Find the new centroids with the purified traces
        x0_filtered = np.mean(traces_0[:, :, 0], axis=1)
        y0_filtered = np.mean(traces_0[:, :, 1], axis=1)
        x1_filtered = np.mean(traces_1[:, :, 0], axis=1)
        y1_filtered = np.mean(traces_1[:, :, 1], axis=1)
        one_indices[one] = indices_1
        
        if plot:
            print('New # traces:%i; old: %i'%(traces_0.shape[0] + traces_1.shape[0], i0.shape[0] + i1.shape[0]))
        
        new_i0 = np.mean(traces_0[:, :, 0], axis=1)
        new_q0 = np.mean(traces_0[:, :, 1], axis=1)
        new_i1 = np.mean(traces_1[:, :, 0], axis=1)
        new_q1 = np.mean(traces_1[:, :, 1], axis=1)

        # Found correct traces for ground and excited states
        # Now to find traces corresponding to 1->0 relaxations and 0->1 excitations
        # Also need to distinguish between initialization errors and relaxations/excitations
        # Find ground truth (0/1) traces

        ground_0_trace = np.mean(traces_0, axis=0)
        ground_1_trace = np.mean(traces_1, axis=0)

        # Relaxation traces vs. |2> state traces:
        # |2> traces are generally in a different direction than relaxation traces;
        # Vector of relaxation trace -> ground state trace
        # Use centroid of ground state cluster with the incorrect |1> trace to determine
        # vector direction. Then if direction is similar to the vector joining the two
        # Centroids (0, 1), then it is most likely a relaxation trace. 

        incorrect_traces_0 = traces[np.where(y==zero)[0], :, :]
        incorrect_traces_1 = traces[np.where(y==one)[0], :, :]
        incorrect_traces_0 = incorrect_traces_0[np.where(distance(i0, q0, x0, y0) >= rscale * radius)[0], :, :]
        incorrect_traces_1 = incorrect_traces_1[np.where(distance(i1, q1, x1, y1) >= rscale * radius)[0], :, :]

        # Find traces showing relaxation by evaluation if the mean of the trace lies in
        # the circle corresponding to the ground state
        incorrect_i1 = np.mean(incorrect_traces_1[:, :, 0], axis=1)
        incorrect_q1 = np.mean(incorrect_traces_1[:, :, 1], axis=1)
        relax_traces_1 = incorrect_traces_1[np.where(distance(incorrect_i1, incorrect_q1, x0, y0) <= rscale * radius)[0], :, :]
        relax_trace = np.mean(relax_traces_1, axis=0)
        ket_2_traces = incorrect_traces_1[np.where(distance(incorrect_i1, incorrect_q1, x0, y0) > rscale * radius)[0], :, :]
        ket_2_trace = np.mean(ket_2_traces, axis=0)
        excitation_traces_0 = incorrect_traces_0
        excitation_trace = np.mean(excitation_traces_0, axis=0)

        data = {}
        data['gnd_0'] = ground_0_trace
        data['gnd_1'] = ground_1_trace
        data['relax'] = relax_trace
        data['ket2'] = ket_2_traces
        data['excite'] = excitation_trace
        data['mean_0'] = tuple((x0, y0))
        data['mean_1'] = tuple((x1, y1))
        data['mean_0_filtered'] = tuple((np.mean(x0_filtered), np.mean(y0_filtered)))
        data['mean_1_filtered'] = tuple((np.mean(x1_filtered), np.mean(y1_filtered)))
        data['traces_relax'] = relax_traces_1
        data['traces_0'] = traces_0
        data['traces_1'] = traces_1
        data['traces_excite'] = incorrect_traces_0
        qubit_traces[qubit] = data

        if plot:
            fig, axs = plt.subplots(1, 2, figsize=(27, 9), constrained_layout=True, sharex='all', sharey='all')
            ax = axs[0]
            ax.scatter(i0, q0, label='0', alpha=1)
            ax.scatter(i1, q1, label='1', alpha=0.1)
            ax.scatter(x0, y0, color='black')
            ax.scatter(x1, y1, color='black')
            ax.scatter(np.mean(x0_filtered), np.mean(y0_filtered), color='crimson')
            ax.scatter(np.mean(x1_filtered), np.mean(y1_filtered), color='crimson')
            ax.plot([x0, x1], [y0, y1], color='black', linestyle='--')
            ax.scatter(midx, midy, color='crimson')
            ax.legend()
            ax.set_title('Original')
            ax = axs[1]
            ax.scatter(new_i0, new_q0, label='0')
            ax.scatter(new_i1, new_q1, label='1', alpha=0.1)
            ax.legend()
            ax.set_title('After filtering')
            plt.suptitle('Qubit %i'%(qubit))
            plt.show()
            print("Correct 0 traces: %i"%(traces_0.shape[0]))
            print("Correct 1 traces: %i"%(traces_1.shape[0]))
            print("Qubit relaxation traces: %i"%(relax_traces_1.shape[0]))
            print("|2> traces: %i"%(ket_2_traces.shape[0]))
            print("Excitation traces: %i"%(excitation_traces_0.shape[0]))

    indices_y = np.zeros(indices_0.shape)
    indices_1 = np.array([])
    for key in one_indices.keys():
        indices_y = np.hstack((indices_y, key * np.ones(one_indices[key].shape)))
        indices_1 = np.hstack((indices_1, one_indices[key]))
    filtered_indices = tuple((indices_0, one_indices, indices_1, indices_y))
    return qubit_traces, filtered_indices

In [ ]:
traces_info, _ = get_traces(plot=True)

### Visualization

In [ ]:
def plot_data_traces(traces_info, qubit):
    qubit_traces_info = traces_info[qubit]
    ground_0_trace = qubit_traces_info['gnd_0']
    ground_1_trace = qubit_traces_info['gnd_1']
    relax_trace = qubit_traces_info['relax']

    plt_ground_0_trace = ground_0_trace.reshape(-1, 25, ground_0_trace.shape[1]).mean(axis = 1)
    plt_ground_1_trace = ground_1_trace.reshape(-1, 25, ground_1_trace.shape[1]).mean(axis = 1)
    plt_relax_trace = relax_trace.reshape(-1, 25, relax_trace.shape[1]).mean(axis = 1)


    plt.plot(plt_ground_0_trace[:, 0], plt_ground_0_trace[:, 1], label='0')
    plt.plot(plt_ground_1_trace[:, 0], plt_ground_1_trace[:, 1], label='1')
    plt.plot(plt_relax_trace[:, 0], plt_relax_trace[:, 1], label='1->0 relaxation')
    plt.xlabel('I (In-Phase)')
    plt.ylabel('Q (Quadrature)')
    plt.legend()
    plt.show()

plot_data_traces(traces_info, 3)

In [ ]:
def get_mf(traces_0, traces_1):
    traces_0 = traces_0.reshape(-1, traces_0.shape[1] * traces_0.shape[2])
    traces_1 = traces_1.reshape(-1, traces_1.shape[1] * traces_1.shape[2])
    # traces_0 generally more than traces_1 traces. 
    # Do random sampling from the trace_0 set
    mf = None
    if traces_1.shape[0] > traces_0.shape[0]:
        indices = np.random.choice(traces_1.shape[0], traces_0.shape[0], replace=False)
        mf = np.mean(traces_0 - traces_1[indices], axis=0) / np.var(traces_0 - traces_1[indices], axis=0)
    else:
        indices = np.random.choice(traces_0.shape[0], traces_1.shape[0], replace=False)
        mf = np.mean(traces_0[indices] - traces_1, axis=0) / np.var(traces_0[indices] - traces_1, axis=0)
    
    filtered = np.sum(np.multiply(mf, traces_0), axis=1)
    filtered = np.sort(filtered)
    ind = int(0.995 * filtered.shape[0])
    threshold = filtered[ind]

    return mf, threshold

def plot_matched_filter(zero_traces, one_traces, title='Matched Filter'):
    mf, thresh = get_mf(zero_traces, one_traces) 
    fig, ax = plt.subplots(1, 1, constrained_layout=True, figsize=(6, 4))
    ax.plot(mf, label='MF Envelope')
    ax.legend(fontsize=20)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(title)

qubit =3
qubit_traces_info = traces_info[qubit]
traces_0 = qubit_traces_info['traces_0']
traces_1 = qubit_traces_info['traces_1']
traces_relax = qubit_traces_info['traces_relax']

plot_matched_filter(traces_0, traces_1, title='Plotting matched filter for zero traces vs one traces')
plot_matched_filter(traces_relax, traces_0, title='Plotting matched filter for zero traces vs relaxation traces')

#### Some helper classes for defining preclassifiers for detecting relaxations

In [ ]:
class preclassifier(demodulation):
    # Can pass train-test ratio and other parameters through constructor later.
    def __init__(self, radius_scale=1) -> None:
        self.filtered_indices = None
        self.rscale = radius_scale
        self.trace_classes = None

    def distance(self, x0, y0, x1, y1):
        return np.sqrt((x0 - x1)**2 + (y0 - y1)**2)
    
    def fit(self):
        qubit_traces, indices_filtered = get_traces()
        self.filtered_indices = indices_filtered
        self.trace_classes = qubit_traces
        return

    def predict(self, data, num_qubits=5):
        data = np.array(data)
        temp = None
        if len(data.shape) == 3:
            indices_0 = self.filtered_indices[0]
            indices_1 = self.filtered_indices[2]
            y = self.filtered_indices[3]
            one_indices = self.filtered_indices[1]
            temp = data[indices_0.astype(int), :, :]
            temp = np.vstack((temp, data[indices_1.astype(int), :, :]))
            return temp, y
        else:
            print('Please pass a dataset of the correct shape.')
            return -1
        return

    def get_traces(self):
        return self.trace_classes

In [ ]:
class relaxation_mf_classifier(preclassifier):
    def __init__(self) -> None:
        self.envelopes = []
        self.thresholds = []
        pass

    def fit(self, trace_classes, num_qubits=5, boxcars=None):
        for qubit in range(1, 1 + num_qubits):
            # Distinguish between |0> and |1> -> |0> traces.
            relaxation_traces = trace_classes[qubit]['traces_relax']
            zero_traces = trace_classes[qubit]['traces_0']
            relaxation_traces = relaxation_traces.reshape(-1, relaxation_traces.shape[1] * relaxation_traces.shape[2])
            zero_traces = zero_traces.reshape(-1, zero_traces.shape[1] * zero_traces.shape[2])
            # zero traces generally more than relaxation traces. 
            # Do random sampling from the zero trace set
            mf = None
            if zero_traces.shape[0] > relaxation_traces.shape[0]:
                indices = np.random.choice(zero_traces.shape[0], relaxation_traces.shape[0], replace=False)
                mf = np.mean(relaxation_traces - zero_traces[indices], axis=0) / np.var(relaxation_traces - zero_traces[indices], axis=0)
            else:
                indices = np.random.choice(relaxation_traces.shape[0], zero_traces.shape[0], replace=False)
                mf = np.mean(relaxation_traces[indices] - zero_traces, axis=0) / np.var(relaxation_traces[indices] - zero_traces, axis=0)
            if boxcars is not None:
                boxcar = np.heaviside((len(mf) - boxcars[qubit - 1] / 50) - np.arange(len(mf)), 1)
                mf = mf * boxcar
            zero_filter = np.sum(np.multiply(mf, zero_traces), axis=1)
            zero_filter = np.sort(zero_filter)
            ind = int(0.995 * zero_filter.shape[0])
            threshold = zero_filter[ind]
            self.envelopes.append(mf)
            self.thresholds.append(threshold)
        return

    def predict(self, num_qubits=5, data_type=0):
        # Use demodulated data dumps
        result = []
        for qubit in range(1, num_qubits + 1):
            dem, y, t = get_data(qubit)
            traces, ys = get_train_val_and_test_set(np.array(dem), np.array(y))
            dem = traces[data_type]
            y = ys[data_type]
            del traces
            traces = dem
            states = np.unique(y)
            traces = traces.reshape(traces.shape[0], -1)
            mf_results = np.empty((states.shape[0], int(traces.shape[0] / states.shape[0])))
            for state in states:
                mf_results[state] = np.array([np.sum(traces[y==state] * self.envelopes[qubit - 1], axis=1)])
            result.append(mf_results)
        result = np.array(result).transpose([1, 2, 0])
        return result

## Running a large NN classifier on the dataset

### Neural Network Architecture

1. **Input Layer**
   - **1000 Neurons**

2. **Hidden Layer 1**
   - **500 Neurons**

3. **Hidden Layer 2**
   - **250 Neurons**

4. **Output Layer**
   - **32 Neurons**



In [ ]:
root_dir = './'

class ADCDataset(T.utils.data.Dataset):

    def __init__(self, data_file):
        all_data = np.load(data_file)
        num_samples_per_state = all_data.shape[1]
        num_basis_state = all_data.shape[0]

        all_labels = []
        for i in range(num_basis_state):
            all_labels.append(np.array([i for _ in range(num_samples_per_state)]))

        all_data = all_data.reshape((num_basis_state * num_samples_per_state, -1))
        all_data = all_data[:, :1000]  # only use the first 1000 samples (500 I's and 500 Q's)
        all_labels = np.array(all_labels).reshape((-1))

        self.x_data = T.tensor(all_data, dtype=T.float)
        self.y_data = T.tensor(all_labels, dtype=T.long)

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        if T.is_tensor(idx):
            idx = idx.tolist()
        preds = self.x_data[idx]
        lbl = self.y_data[idx]
        sample = {'predictors': preds, 'target': lbl}

        return sample


def adjust_learning_rate(initial_lr, optimizer, epoch, lr_schedule=[30, 60, 90]):
    lr = initial_lr
    if epoch >= lr_schedule[0]:
        lr *= 0.1

    if epoch >= lr_schedule[1]:
        lr *= 0.1

    if epoch >= lr_schedule[2]:
        lr *= 0.1

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr


def inference(model, dl):
    model.eval()
    all_scores = []
    all_labels = []
    s = T.nn.Softmax(dim=-1)
    with T.no_grad():
        for (batch_idx, batch) in enumerate(dl):
            X = batch['predictors']
            Y = batch['target']
            oupt = model(X)
            oupt = s(oupt)

            all_scores.extend(oupt.cpu().numpy())
            all_labels.extend(Y.cpu().numpy())

    model.train()
    return np.array(all_scores), np.array(all_labels)


def accuracy(model, dl):
    all_preds, all_labels = inference(model, dl)
    # logger.debug(all_preds.shape)

    pred_indices = np.argmax(all_preds, axis=-1)
    cumulative_acc = np.sum(pred_indices == all_labels) / len(all_labels)

    acc_per_qubit = []
    for _ in range(5):
        pred_qubit = pred_indices % 2
        label_qubit = all_labels % 2
        acc_per_qubit.append(np.sum(pred_qubit == label_qubit) / len(label_qubit))
        pred_indices = pred_indices >> 1
        all_labels = all_labels >> 1

    return cumulative_acc, acc_per_qubit

In [ ]:
def load_data_all(NUM_TRAIN_VAL=3000, NUM_TEST = 7000, NUM_VAL_RATIO = 0.35):
    NUM_VAL = int(NUM_TRAIN_VAL * NUM_VAL_RATIO)
    NUM_TRAIN = NUM_TRAIN_VAL - NUM_VAL
    TOTAL_TRACES = NUM_TRAIN_VAL + NUM_TEST
    print('length of training set: {}'.format(NUM_TRAIN))
    print('length of val set: {}'.format(NUM_VAL))
    print('length of test set: {}'.format(NUM_TEST))
    
    train_set = []
    val_set = []
    test_set = []

    data = np.load('all_traces_10k.npy')
    print(data.shape)
    
    for basis_state_data in data:
        random.shuffle(basis_state_data)
        train_set.append(basis_state_data[:NUM_TRAIN])
        val_set.append(basis_state_data[NUM_TRAIN:NUM_TRAIN_VAL])
        test_set.append(basis_state_data[NUM_TRAIN_VAL:])

    train_set = np.array(train_set)
    print('Train set: ', train_set.shape)
    val_set = np.array(val_set)
    print('Val set: ', val_set.shape)
    test_set = np.array(test_set)
    print('Test set: ', test_set.shape)


    os.makedirs('split_data', exist_ok=True)
    np.save('split_data/train.npy', train_set)
    np.save('split_data/val.npy', val_set)
    np.save('split_data/test.npy', test_set)
    
    return train_set, val_set, test_set

def data_load(num_Q=5, rep_seq=False):

    ## structure: [QB5, QB4, QB3, QB2, QB1]
    data = h5py.File('ADC_10kData', 'r')
    num_records 	= int(1e4) #number of repeated acquisitions

    print('loading Ch1')
    Ch1 		= data['Data'][:, :, 0]  # shape (1600000, 1473) = (32 * num_records, num_samples)
    print('loading Ch2')
    Ch2 		= data['Data'][:, :, 1]  # shape (1600000, 1473) = (32 * num_records, num_samples)

    ## statistics
    num_samples_raw = Ch1.shape[1] #number of samples per acquisition
    print('num_samples_raw: ', num_samples_raw)

    ## data preparation
    DataRaw = np.zeros((2**num_Q, num_records, num_samples_raw, 2))
    DataRaw[:, :, :, 0] = np.reshape(Ch1, (2 ** num_Q, num_records, num_samples_raw))
    DataRaw[:, :, :, 1] = np.reshape(Ch2, (2 ** num_Q, num_records, num_samples_raw))
    print('DataRaw shape: ', DataRaw.shape)
    
    np.save('all_traces_10k.npy', DataRaw)
    data.close()
    del Ch1, Ch2
    return

data_load(num_Q=5, rep_seq=False)
train_set, val_set, test_set = load_data_all(NUM_TRAIN_VAL=3000, NUM_TEST=7000, NUM_VAL_RATIO=0.35)

In [ ]:
class Net_baseline(T.nn.Module):
    def __init__(self):
        super(Net_baseline, self).__init__()
        self.hid1 = T.nn.Linear(1000, 500)
        self.hid2 = T.nn.Linear(500, 250)
        self.oupt = T.nn.Linear(250, 32)

        T.nn.init.xavier_uniform_(self.hid1.weight)
        T.nn.init.zeros_(self.hid1.bias)
        T.nn.init.xavier_uniform_(self.hid2.weight)
        T.nn.init.zeros_(self.hid2.bias)
        T.nn.init.xavier_uniform_(self.oupt.weight)
        T.nn.init.zeros_(self.oupt.bias)
        self.relu = T.nn.ReLU()

    def forward(self, x):
        z = self.hid1(x)
        z = self.relu(z)
        z = self.hid2(z)
        z = self.relu(z)
        z = self.oupt(z)
        return z
# ----------------------------------------------------------

def train_baseline():
    # 0. get started
    T.manual_seed(1)
    np.random.seed(1)

    # 1. create Dataset and DataLoader objects

    train_ds = ADCDataset(os.path.join(root_dir, 'split_data/train.npy'))
    val_ds = ADCDataset(os.path.join(root_dir, 'split_data/val.npy'))
    test_ds = ADCDataset(os.path.join(root_dir, 'split_data/test.npy'))

    bat_size = 512
    train_ldr = T.utils.data.DataLoader(train_ds, batch_size=bat_size, shuffle=True)
    val_ldr = T.utils.data.DataLoader(val_ds, batch_size=bat_size, shuffle=False)
    test_ldr = T.utils.data.DataLoader(test_ds, batch_size=bat_size, shuffle=False)

    # 2. create neural network
    # Creating 1000-500-250-32 binary NN classifier ")
    net = Net_baseline()  # .to(device)

    # 3. train network
    net = net.train()  # set training mode
    lrn_rate = 0.0001
    loss_obj = T.nn.CrossEntropyLoss()  # cross entropy
    optimizer = T.optim.Adam(net.parameters(), lr=lrn_rate)
    max_epochs = 100
    ep_log_interval = 1


    best_acc = -1

    path = os.path.join(".", "checkpoints/1000_points")
    os.makedirs(path, exist_ok=True)

    for epoch in range(0, max_epochs):
        epoch_loss = 0.0  # for one full epoch

        lr = adjust_learning_rate(lrn_rate, optimizer, epoch)

        for (batch_idx, batch) in enumerate(train_ldr):
            X = batch['predictors']
            Y = batch['target']
            oupt = net(X)

            loss_val = loss_obj(oupt, Y)
            epoch_loss += loss_val.item()

            optimizer.zero_grad()
            loss_val.backward()
            optimizer.step()
        if epoch % ep_log_interval == 0:
            print("epoch = %4d   loss = %0.4f  lr = %0.4f" % \
                  (epoch, epoch_loss, lr))

        acc_train, acc_train_per_qubit = accuracy(net, train_ldr)
        acc_val, acc_val_per_qubit = accuracy(net, val_ldr)

        if acc_val >= best_acc:
            best_acc = acc_val
            T.save(net.state_dict(), os.path.join(path, 'best_epoch.pth'))

    print("Finished  training")
    print("Best acc on val dataset: {}".format(best_acc))

    net.load_state_dict(T.load(os.path.join(path, 'best_epoch.pth')))
    acc_test, acc_test_per_qubit = accuracy(net, test_ldr)
    print("Acc on test dataset: {}".format(acc_test))
    print("Acc per qubit: {}".format(acc_test_per_qubit))

train_baseline()

### Running HERQULES


In [ ]:
from matched_filter import search_matched_filter_for_all_qubits, matched_filter_preprocess, search_matched_filter_for_all_qubits_preclass, matched_filter_preprocess_demux, search_matched_filter_for_all_qubits_demux


hf = h5py.File('./ADC_10kData', 'r')
train_semi_sup_data = hf.get('Data')

os.makedirs('accuracy', exist_ok=True)
os.makedirs('stats', exist_ok=True)
os.makedirs('logs', exist_ok=True)

#root_dir = '/nobackup/readout_data/new'
root_dir = '.'
# root_dir ='.'

def mf_demux_data_prep():
    # Prepare data
    data_train = []
    data_val = []
    data_test = []
    for qubit in range(1, 6):
        traces, y, t = get_data(qubit)
        traces, y = get_train_val_and_test_set(np.array(traces), np.array(y))
        data = traces[0].reshape(32, int(traces[0].shape[0] / 32), traces[0].shape[1], traces[0].shape[2])
        data_train.append(data)
        data = traces[1].reshape(32, int(traces[1].shape[0] / 32), traces[1].shape[1], traces[1].shape[2])
        data_val.append(data)
        data = traces[2].reshape(32, int(traces[2].shape[0] / 32), traces[2].shape[1], traces[2].shape[2])
        data_test.append(data)
    return data_train, data_val, data_test

class MFOutputDataset(T.utils.data.Dataset):

    def __init__(self, all_data):
        num_samples_per_state = all_data.shape[1]
        num_basis_state = all_data.shape[0]

        all_labels = []
        for i in range(num_basis_state):
            all_labels.append(np.array([i for _ in range(num_samples_per_state)]))

        all_data = all_data.reshape((num_basis_state * num_samples_per_state, -1))
        all_labels = np.array(all_labels).reshape((-1))

        self.x_data = T.tensor(all_data, dtype=T.float)
        self.y_data = T.tensor(all_labels, dtype=T.long)

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        if T.is_tensor(idx):
            idx = idx.tolist()
        preds = self.x_data[idx]
        lbl = self.y_data[idx]
        sample = {'predictors': preds, 'target': lbl}

        return sample


def adjust_learning_rate(initial_lr, optimizer, epoch, lr_schedule=[30, 60, 90]):
    lr = initial_lr
    if epoch >= lr_schedule[0]:
        lr *= 0.1

    if epoch >= lr_schedule[1]:
        lr *= 0.1

    if epoch >= lr_schedule[2]:
        lr *= 0.1

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr


def inference(model, dl):
    model.eval()
    all_scores = []
    all_labels = []
    s = T.nn.Softmax(dim=-1)
    with T.no_grad():
        for (batch_idx, batch) in enumerate(dl):
            X = batch['predictors']
            Y = batch['target']
            oupt = model(X)
            oupt = s(oupt)

            all_scores.extend(oupt.cpu().numpy())
            all_labels.extend(Y.cpu().numpy())

    model.train()
    return np.array(all_scores), np.array(all_labels)


def accuracy(model, dl):
    all_preds, all_labels = inference(model, dl)
    pred_indices = np.argmax(all_preds, axis=-1)
    cumulative_acc = np.sum(pred_indices == all_labels) / len(all_labels)
    data = {'preds':pred_indices, 'labels':all_labels}
    with open('mf_rmf_nn_train.pkl', 'wb') as file:
        pickle.dump(data, file)
    acc_per_qubit = []
    for i in range(5):
        pred_qubit = pred_indices % 2
        label_qubit = all_labels % 2
        acc_per_qubit.append(np.sum(pred_qubit == label_qubit) / len(label_qubit))
        pred_indices = pred_indices >> 1
        all_labels = all_labels >> 1

    return cumulative_acc, acc_per_qubit


class Net(T.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hid1 = T.nn.Linear(5, 10)
        self.hid2 = T.nn.Linear(10, 20)
        self.oupt = T.nn.Linear(20, 32)

        T.nn.init.xavier_uniform_(self.hid1.weight)
        T.nn.init.zeros_(self.hid1.bias)
        T.nn.init.xavier_uniform_(self.hid2.weight)
        T.nn.init.zeros_(self.hid2.bias)
        T.nn.init.xavier_uniform_(self.oupt.weight)
        T.nn.init.zeros_(self.oupt.bias)
        self.relu = T.nn.ReLU()

    def forward(self, x):
        z = self.hid1(x)
        z = self.relu(z)
        z = self.hid2(z)
        z = self.relu(z)
        z = self.oupt(z)
        return z

class Net_rmf(T.nn.Module):
    def __init__(self):
        super(Net_rmf, self).__init__()
        self.hid1 = T.nn.Linear(10, 10)
        self.hid2 = T.nn.Linear(10, 20)
        self.oupt = T.nn.Linear(20, 32)

        T.nn.init.xavier_uniform_(self.hid1.weight)
        T.nn.init.zeros_(self.hid1.bias)
        T.nn.init.xavier_uniform_(self.hid2.weight)
        T.nn.init.zeros_(self.hid2.bias)
        T.nn.init.xavier_uniform_(self.oupt.weight)
        T.nn.init.zeros_(self.oupt.bias)
        self.relu = T.nn.ReLU()

    def forward(self, x):
        z = self.hid1(x)
        z = self.relu(z)
        z = self.hid2(z)
        z = self.relu(z)
        z = self.oupt(z)
        return z


# ----------------------------------------------------------

def train(run_pre_filter=False,
          run_semi_sup=True,
          run_rmf = True,
          train_data_ratio = 1, dur=1000):
    
    # 0. get started
    T.manual_seed(1)
    np.random.seed(1)

    # 1. create Dataset and DataLoader objects
    
    semi_sup_classifier = preclassifier()
    rmf = relaxation_mf_classifier()
    train_rmf_data = None
    val_rmf_data = None
    test_rmf_data = None
    semi_sup_classifier.fit()
    if run_semi_sup:
        filtered_train_data, filtered_labels = semi_sup_classifier.predict(train_semi_sup_data)
    fast_readout=20 - int(dur / 50)
    boxcars = [1, 1, 9, 2, 9]
    print('Boxcars: ', boxcars)
    trace_classes = None
    if run_rmf:
        ##Get relaxtional matched filter data
        trace_classes = semi_sup_classifier.get_traces()
        rmf.fit(trace_classes, boxcars=boxcars)
        #data_type => 0 for train, 1 for val, 2 for test
        train_rmf_data = rmf.predict(data_type=0)
        val_rmf_data = rmf.predict(data_type=1)
        test_rmf_data = rmf.predict(data_type=2)
            
    best_bc = boxcars
    #best_bc=[0, 0, 0, 0, 0]

    demux_data_train, demux_data_val, demux_data_test = mf_demux_data_prep()
    print(np.array(demux_data_train).shape)

    if run_pre_filter:
        mf_envelopes, _ = search_matched_filter_for_all_qubits(filtered_train_data, best_bc=best_bc)
    elif run_semi_sup:
        mf_envelopes, _ = search_matched_filter_for_all_qubits_preclass(filtered_train_data, filtered_labels, best_bc=best_bc)
    else:
        #mf_envelopes, _ = search_matched_filter_for_all_qubits(train_data, best_bc=best_bc)
        mf_envelopes, _ = search_matched_filter_for_all_qubits_demux(demux_data_train, best_bc=best_bc)

    no_mf = False

    if no_mf and run_rmf:
        train_ds = MFOutputDataset(train_rmf_data)
        val_ds = MFOutputDataset(val_rmf_data)
        test_ds = MFOutputDataset(test_rmf_data)
    elif run_rmf:
        train_ds =  MFOutputDataset(np.concatenate((matched_filter_preprocess_demux(demux_data_train, mf_envelopes), train_rmf_data), axis=2))
        val_ds = MFOutputDataset(np.concatenate((matched_filter_preprocess_demux(demux_data_val, mf_envelopes), val_rmf_data), axis=2))
        test_ds = MFOutputDataset(np.concatenate((matched_filter_preprocess_demux(demux_data_test, mf_envelopes), test_rmf_data), axis=2))
    else:
        train_ds = MFOutputDataset(matched_filter_preprocess_demux(demux_data_train, mf_envelopes))
        val_ds = MFOutputDataset(matched_filter_preprocess_demux(demux_data_val, mf_envelopes))
        test_ds = MFOutputDataset(matched_filter_preprocess_demux(demux_data_test, mf_envelopes))


    bat_size = 512
    train_ldr = T.utils.data.DataLoader(train_ds, batch_size=bat_size, shuffle=True)
    val_ldr = T.utils.data.DataLoader(val_ds, batch_size=bat_size, shuffle=False)
    test_ldr = T.utils.data.DataLoader(test_ds, batch_size=bat_size, shuffle=False)

    # 2. create neural network
    # Creating 10-50-25-32 binary NN classifier
    if run_rmf and no_mf == False:
        net = Net_rmf()  # .to(device)
    else:
        net = Net()

    # 3. train network
    net = net.train()  # set training mode
    lrn_rate = 0.01
    loss_obj = T.nn.CrossEntropyLoss()  # cross entropy
    optimizer = T.optim.Adam(net.parameters(), lr=lrn_rate)
    max_epochs = 100
    ep_log_interval = 1



    best_acc = -1
    acc_train_itr, acc_val_itr = [], []
    path = os.path.join(".", "checkpoints/mf_nn")
    os.makedirs(path, exist_ok=True)
    with T.profiler.profile(
        schedule=T.profiler.schedule(wait=1, warmup=1, active=3, repeat=2),
        on_trace_ready=T.profiler.tensorboard_trace_handler('./logs/tb_nn'),
        record_shapes=True,
        profile_memory=True,
        with_stack=True
) as prof:
        for epoch in range(0, max_epochs):
            epoch_loss = 0.0  # for one full epoch

            lr = adjust_learning_rate(lrn_rate, optimizer, epoch)

            for (batch_idx, batch) in enumerate(train_ldr):
                X = batch['predictors']
                Y = batch['target']
                # print(X)
                oupt = net(X)

                loss_val = loss_obj(oupt, Y)
                epoch_loss += loss_val.item()

                optimizer.zero_grad()
                loss_val.backward()
                optimizer.step()
                prof.step()
            if epoch % ep_log_interval == 0:
                print("epoch = %4d   loss = %0.4f  lr = %0.4f" % \
                    (epoch, epoch_loss, lr))

            acc_train, acc_train_per_qubit = accuracy(net, train_ldr)
            acc_train_itr.append(acc_train_per_qubit)
            acc_val, acc_val_per_qubit = accuracy(net, val_ldr)
            acc_train_itr.append(acc_val_per_qubit)

            if acc_val >= best_acc:
                best_acc = acc_val
                T.save(net.state_dict(), os.path.join(path, 'best_epoch.pth'))

    print("Finished  training")
    print("Best acc on val dataset: {}".format(best_acc))

    net.load_state_dict(T.load(os.path.join(path, 'best_epoch.pth')))
    acc_test, acc_test_per_qubit = accuracy(net, test_ldr)
    print("Acc on test dataset: {}".format(acc_test))
    print("Acc per qubit: {}".format(acc_test_per_qubit))
    print("Acc per qubit: {}".format(acc_test_per_qubit))
    return acc_test_per_qubit
    

train(run_semi_sup=False, run_pre_filter=False, run_rmf=True)
